# IBM Applied Data Science Capstone Course by Coursera

## Week 5 Capstone project

### Exploring avenues for Opening a Bakery in Mumbai

### 1. Install all the required Libraries

In [43]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [44]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [46]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [47]:
# library to handle data in a vectorized manner
import numpy as np

# library for data analsysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

# library to handle requests
import requests
from tqdm import tqdm

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print("Additinal libraries downloaded")

print('Libraries imported.')

Additinal libraries downloaded
Libraries imported.


### 2. Get data for Neighbourhoods in Mumbai & Page scraping

In [48]:
URL = "https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


### 3. Creating a dataframe for the Mumbai data

In [49]:
# create a list to store neighborhood data
neighborhoodList = []

In [50]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [51]:
# create a new DataFrame from the list
Mum_df = pd.DataFrame({"Neighborhood": neighborhoodList})

Mum_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [52]:
# print the number of rows of the dataframe
Mum_df.shape

(42, 1)

### 4. Get the geographical Locations

In [53]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [54]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in Mum_df["Neighborhood"].tolist() ]

In [55]:
coords

[[19.118459378296492, 72.84176321065843],
 [19.042830000000038, 72.92734000000007],
 [19.062940000000026, 72.92663000000005],
 [19.054370000000063, 72.84017000000006],
 [19.145560000000046, 72.94856000000004],
 [19.229360000000042, 72.85751000000005],
 [19.208660000000066, 72.82612000000006],
 [19.06218000000007, 72.90241000000003],
 [19.250030000000038, 72.85907000000003],
 [19.224690000000066, 72.86605000000003],
 [19.212750000000028, 73.08324000000005],
 [19.00426978951916, 72.85579215153318],
 [19.08652321008152, 72.90900774216628],
 [19.164550000000077, 72.84946000000008],
 [18.959290000000067, 72.83108000000004],
 [19.137920000000065, 72.84941000000003],
 [19.014920000000075, 72.84522000000004],
 [18.953937419095155, 72.82036732944775],
 [19.21198153260436, 72.83757275783374],
 [19.131380000000036, 72.93568000000005],
 [19.127580000000023, 72.82539000000008],
 [19.064980000000048, 72.88069000000007],
 [19.21094000000005, 72.84137000000004],
 [19.048530000000028, 72.93222000000003

In [56]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [57]:
# merge the coordinates into the original dataframe
Mum_df['Latitude'] = df_coords['Latitude']
Mum_df['Longitude'] = df_coords['Longitude']

In [58]:
# check the neighborhoods and the coordinates
print(Mum_df.shape)
Mum_df

(42, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [59]:
# save the DataFrame as CSV file
Mum_df.to_csv("Mum_df.csv", index=False)

### 5. Explore the Mumbai data in a map with neighborhoods superimposed on top

In [60]:
address = 'Mumbai, India'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Mumbai, India are 18.9387711, 72.8353355.


In [61]:
# create map of Mumbai using latitude and longitude values
map_Mumbai = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(Mum_df['Latitude'], Mum_df['Longitude'],Mum_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Mumbai)  
    
map_Mumbai

In [62]:
# save the map as HTML file
map_Mumbai.save('map_Mumbai.html')

### 6. Configure Foursquare access

In [63]:
CLIENT_ID = 'PUATVQBZHYWEFYTO0VRGZRKERUV1MU1S1YGLATFQHIRH5WQB' # your Foursquare ID
CLIENT_SECRET = 'IOPFUZ0FLKMMUJYBL3E0HB5RLSYVLGWSLI4ALJX4URYOLSXE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PUATVQBZHYWEFYTO0VRGZRKERUV1MU1S1YGLATFQHIRH5WQB
CLIENT_SECRET:IOPFUZ0FLKMMUJYBL3E0HB5RLSYVLGWSLI4ALJX4URYOLSXE


### 7. Get the top 100 venues that are within a radius of 1000 meters.

In [64]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Mum_df['Latitude'], Mum_df['Longitude'], Mum_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [65]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1078, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,Cafe Alfa,19.119667,72.843560,Indian Restaurant
3,Andheri,19.118459,72.841763,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
4,Andheri,19.118459,72.841763,Temptations,19.113767,72.841337,Ice Cream Shop


In [66]:
##Let's check how many venues were returned for each neighorhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,37,37,37,37,37,37
Anushakti Nagar,6,6,6,6,6,6
Baiganwadi,7,7,7,7,7,7
Bandra,65,65,65,65,65,65
Bhandup,8,8,8,8,8,8
Borivali,49,49,49,49,49,49
Charkop,8,8,8,8,8,8
Chembur,42,42,42,42,42,42
Dahisar,24,24,24,24,24,24


In [67]:
##Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 150 uniques categories.


In [68]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Indian Restaurant', 'Falafel Restaurant',
       'Ice Cream Shop', 'Coffee Shop', 'Pub', 'Sandwich Place',
       'Chinese Restaurant', 'Snack Place', 'Fast Food Restaurant',
       'Vegetarian / Vegan Restaurant', 'Department Store', 'Café',
       'Pizza Place', 'Bagel Shop', 'Restaurant', 'Food Court',
       'Bookstore', 'Nightclub', 'Gym', 'Hotel Bar', 'Asian Restaurant',
       'Bus Station', 'Train Station', 'ATM', 'Adult Boutique',
       'Fried Chicken Joint', 'Lounge', 'Gourmet Shop',
       'Korean Restaurant', 'Sports Club', 'Park', 'BBQ Joint',
       'Seafood Restaurant', 'Indie Movie Theater', 'Convenience Store',
       'Italian Restaurant', 'Clothing Store', 'Bar', "Women's Store",
       'Hookah Bar', "Men's Store", 'Theater', 'Steakhouse',
       'Latin American Restaurant', 'Burger Joint', 'Pier', 'Multiplex',
       'Tunnel', 'Smoke Shop'], dtype=object)

In [69]:
# check if the results contain "Bakery"
"Bakery" in venues_df['VenueCategory'].unique()

True

### 8. Analyze each neighbourhood

In [70]:
# one hot encoding
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(1078, 151)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Station,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Department Store,Design Studio,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Historic Site,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Opera House,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pub,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Tea Room,Thai Restaurant,Theater,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [71]:
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped

(42, 151)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Buffet,Burger Joint,Bus Station,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Department Store,Design Studio,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Historic Site,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Opera House,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pub,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Tea Room,Thai Restaurant,Theater,Trail,Train,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.027027,0.027027,0.000000,0.000000,0.000000,0.0,0.000000,0.027027,0.000000,0.000000,0.000,0.000000,0.000000,0.054054,0.000000,0.027027,0.000000,0.000000,0.054054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000,0.054054,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.054054,0.270270,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000,0.054054,0.000000,0.000000,0.027027,0.000000,0.027027,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.027027,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.054054,0.000000,0.000000,0.000000
1,Anushakti Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [72]:
len(mum_grouped[mum_grouped["Bakery"] > 0])

16

### 9. Create a new DataFrame for Bakery data only

In [73]:
mum_mall = mum_grouped[["Neighborhoods","Bakery"]]

In [74]:
mum_mall

,Neighborhoods,Bakery
0,Andheri,0.027027
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.000000
4,Bhandup,0.000000
5,Borivali,0.020408
6,Charkop,0.125000
7,Chembur,0.023810
8,Dahisar,0.000000
9,Devipada,0.000000


### 10. Cluster neighbourhoods

In [75]:
# set number of clusters
kclusters = 3

mum_clustering = mum_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 2, 2, 0, 1, 0, 2, 2], dtype=int32)

In [76]:
# add clustering labels
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mum_merged = mum_mall.copy()

# add clustering labels
mum_merged["Cluster Labels"] = kmeans.labels_

In [77]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Bakery,Cluster Labels
0,Andheri,0.027027,0
1,Anushakti Nagar,0.000000,2
2,Baiganwadi,0.000000,2
3,Bandra,0.000000,2
4,Bhandup,0.000000,2


In [78]:
# merge mumbai_grouped with mumbai_data to add latitude/longitude for each neighborhood
mum_merged = mum_merged.join(Mum_df.set_index("Neighborhood"), on="Neighborhood")

print(mum_merged.shape)
mum_merged.head() # check the last columns!

(42, 5)


,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
0,Andheri,0.027027,0,19.118459,72.841763
1,Anushakti Nagar,0.000000,2,19.042830,72.927340
2,Baiganwadi,0.000000,2,19.062940,72.926630
3,Bandra,0.000000,2,19.054370,72.840170
4,Bhandup,0.000000,2,19.145560,72.948560


In [79]:
# sort the results by Cluster Labels
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

(42, 5)


,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
0,Andheri,0.027027,0,19.118459,72.841763
37,Vashi,0.026316,0,19.084650,72.904810
32,"Sion, Mumbai",0.034483,0,19.043410,72.863320
24,"Matharpacady, Mumbai",0.035714,0,18.950694,72.827268
22,Mahavir Nagar (Kandivali),0.068966,0,19.210940,72.841370
40,Western Suburbs (Mumbai),0.043478,0,19.197010,72.827680
18,Kandivali,0.058824,0,19.211982,72.837573
14,Grant Road,0.026316,0,18.959290,72.831080
12,Ghatkopar,0.027778,0,19.086523,72.909008
41,Worli,0.041667,0,19.007440,72.816880


### 11. Let us visualize the clusters

In [80]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [81]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 12. Examine clusters

#### Cluster 0

In [82]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
0,Andheri,0.027027,0,19.118459,72.841763
37,Vashi,0.026316,0,19.084650,72.904810
32,"Sion, Mumbai",0.034483,0,19.043410,72.863320
24,"Matharpacady, Mumbai",0.035714,0,18.950694,72.827268
22,Mahavir Nagar (Kandivali),0.068966,0,19.210940,72.841370
40,Western Suburbs (Mumbai),0.043478,0,19.197010,72.827680
18,Kandivali,0.058824,0,19.211982,72.837573
14,Grant Road,0.026316,0,18.959290,72.831080
12,Ghatkopar,0.027778,0,19.086523,72.909008
41,Worli,0.041667,0,19.007440,72.816880


#### Cluster 1

In [83]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
6,Charkop,0.125,1,19.20866,72.82612
36,Uttan,0.200,1,26.86634,80.93884


#### Cluster 3

In [84]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
29,Pestom sagar,0.000000,2,19.070640,72.902170
30,Seven Bungalows,0.014706,2,19.131342,72.816342
31,Shil Phata,0.000000,2,19.146580,73.040050
2,Baiganwadi,0.000000,2,19.062940,72.926630
33,"Sonapur, Bhandup",0.000000,2,19.163940,72.935440
10,Dombivli,0.000000,2,19.212750,73.083240
34,Thakur village,0.000000,2,19.210200,72.875410
35,Tilak Nagar (Mumbai),0.000000,2,18.996160,72.852790
1,Anushakti Nagar,0.000000,2,19.042830,72.927340
38,Vikhroli,0.000000,2,19.111090,72.927810
